In [1]:
import os
os.chdir("../")

import pytesseract
import easyocr
from PIL import Image

In [2]:
filename = 'data/menues/002.jpg'
filename_text = 'data/menues_text/002.txt'

In [3]:
image = Image.open(filename)
with open(filename_text, 'r') as f:
    text = f.read().replace('\n', ' ').replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", "")
text

'La cuisine de Linnéa & Xsénia Délicieuse Cuisine Internationale  PetitDéjeuner  Œufs et toast $300 Weisswurst et bretzel $500  Déjeuner  Sandwich $600 Cheeseburger ou hamburger $700 Poisson & champignons $800  Dessert  Gâteau au chocolat $400 Salade de fruits $200  Boissons  Thé $100 Limonade $200 Lait $300'

In [4]:
reader = easyocr.Reader(['fr'])
result_ocr = reader.readtext(filename, detail=0) 
print("EasyOCR: \n", "\n".join(result_ocr).replace('\n', ' ').replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", ""))

EasyOCR: 
 La cuisine de Linnéa & Xsénia Délicieuse Cuisine Internationale PetitDéjeuner Oeufs et toast $300 Weisswurst et bretzel $500 Déjeuner Sanduich $600 CheeSeburger ou hamburger $700 Poisson & champignons $800 Dessert Gâteau au chocolat $400 Salade de fruits $200 Boissons Thé $100 Limonade $200 Lat $300 4 8 C886 0886g 46866 88826


In [5]:
result_tes = pytesseract.image_to_string(image, lang='fra')
print("Tesseract: \n", result_tes.replace('\n', ' ').replace(".", "").replace(",", "").replace(":", "").replace(";", "").replace("!", "").replace("?", "").replace("…", "").replace("|", "").replace("/", "").replace("-", ""))

Tesseract: 
 La CuiSine de Linnéa & XSéna Délicieuse Cuisine TInlerndionale  Petit Déjettner » Oeufs et toast $3 00 LL D ssunsé et Êretze $5 00  Déjeuner Cheesebutrger où hamburger $700  Poisson & champignons $800  Dessert  Gétlectt act chocol $400 Salade de fruits $200   Boissons î Thé $100 Een 8200 >   Lait $300  
